In [1]:
# Dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from config import g_key

In [2]:
# import cleaned master crime csv file
master_file = "Data Output/Toronto_Crimes_Occurred_2014_2019.csv"

# create a dataframe from the master data
master_df = pd.read_csv(master_file, index_col=0)
master_df

,event_unique_id,premisetype,ucr_code,ucr_ext,offence,reportedyear,reportedmonth,reportedday,reporteddayofyear,reporteddayofweek,...,occurrenceday,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Neighbourhood,Long,Lat
0,GO-20141756319,Commercial,1430.0,100.0,Assault,2014.0,March,24.0,83.0,Monday,...,24.0,83.0,Monday,1.0,Assault,D42,132,Malvern (132),-79.199081,43.800281
1,GO-20143006885,Other,2120.0,200.0,B&E,2014.0,September,29.0,272.0,Monday,...,27.0,270.0,Saturday,16.0,Break and Enter,D52,76,Bay Street Corridor (76),-79.386383,43.662472
2,GO-20141756802,Commercial,2120.0,200.0,B&E,2014.0,March,24.0,83.0,Monday,...,24.0,83.0,Monday,6.0,Break and Enter,D23,1,West Humber-Clairville (1),-79.612595,43.720406
3,GO-20141760570,Apartment,2120.0,200.0,B&E,2014.0,March,24.0,83.0,Monday,...,24.0,83.0,Monday,15.0,Break and Enter,D33,47,Don Valley Village (47),-79.349121,43.782772
4,GO-20142004859,Commercial,1610.0,210.0,Robbery - Business,2014.0,May,3.0,123.0,Saturday,...,3.0,123.0,Saturday,2.0,Robbery,D11,90,Junction Area (90),-79.458778,43.664490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229107,GO-20192465270,NaN,NaN,NaN,Stabbing,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,homicide,D51,73,Moss Park (73),-79.371033,43.658295
229108,GO-20192490349,NaN,NaN,NaN,Shooting,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,homicide,D22,13,Etobicoke West Mall (13),-79.562935,43.639656
229109,GO-20192503671,NaN,NaN,NaN,Shooting,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,homicide,D43,138,Eglinton East (138),-79.239731,43.741611
229110,GO-20192520736,NaN,NaN,NaN,Shooting,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,homicide,D42,132,Malvern (132),-79.227135,43.810932


### Q2. Which neighborhoods experience the highest and lowest crime rates in Toronto? 
    a. How close were police stations to where the crime occurred?

In [3]:
# ************************************************************************************************
# use master crime dataset to generate heat map of total crimes across Toronto by neighbourhood
# ************************************************************************************************

# configure gmaps figure
gmaps.configure(api_key=g_key)

# Toronto's coordinates for centering the map
toronto_coords = (43.70, -79.33)

# create base map
fig = gmaps.figure(center=toronto_coords, zoom_level=11)
locations = master_df[["Lat", "Long"]]

heatmap_layer = gmaps.heatmap_layer(locations)
fig.add_layer(heatmap_layer)

# adjust layer settings for increased visibility
heatmap_layer.max_intensity = 50
heatmap_layer.point_radius = 4
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# sort the master dataframe by neighbourhood ID and create a dictionary with relevant neighbourhood details
master_df_sorted = master_df.sort_values("Hood_ID")
neighbourhood_crimes_dict = {"Neighbourhood ID": master_df_sorted["Hood_ID"].unique(),
                             "Hood Lat": master_df_sorted.groupby("Hood_ID")["Lat"].mean(),
                             "Hood Lng": master_df_sorted.groupby("Hood_ID")["Long"].mean(),
                             "No. of Crimes": master_df_sorted.groupby("Hood_ID")["Hood_ID"].count()
}
# create new dataframe from the dictionary, add neighbourhood column and sort in total crimes descending order 
neighbourhood_crimes_df = pd.DataFrame(neighbourhood_crimes_dict).set_index("Neighbourhood ID")
neighbourhood_crimes_df["Neighbourhood"] = master_df_sorted["Neighbourhood"].unique()
neighbourhood_crimes_df.sort_values("No. of Crimes", ascending=False, inplace=True)

neighbourhood_crimes_df

,Hood Lat,Hood Lng,No. of Crimes,Neighbourhood
Neighbourhood ID,,,,
77,43.644959,-79.387431,9984,Waterfront Communities-The Island (77)
76,43.657212,-79.383738,8765,Bay Street Corridor (76)
75,43.659622,-79.379598,7641,Church-Yonge Corridor (75)
1,43.721569,-79.597207,5759,West Humber-Clairville (1)
73,43.656880,-79.369331,5502,Moss Park (73)
...,...,...,...,...
12,43.634161,-79.573765,467,Markland Wood (12)
140,43.748982,-79.198807,424,Guildwood (140)
29,43.714195,-79.480897,408,Maple Leaf (29)


In [5]:
# ************************************************************************************************
# find the neighborhoods that experience the highest and lowest crime rates in Toronto
# ************************************************************************************************

# sort the master dataframe by neighbourhood ID and create a dictionary with relevant neighbourhood details
master_df_sorted = master_df.sort_values("Hood_ID")
neighbourhood_crimes_dict = {"Neighbourhood ID": master_df_sorted["Hood_ID"].unique(),
                             "Hood Lat": master_df_sorted.groupby("Hood_ID")["Lat"].mean(),
                             "Hood Lng": master_df_sorted.groupby("Hood_ID")["Long"].mean(),
                             "No. of Crimes": master_df_sorted.groupby("Hood_ID")["Hood_ID"].count()
}
# create new dataframe from the dictionary, add neighbourhood column and sort in total crimes descending order 
neighbourhood_crimes_df = pd.DataFrame(neighbourhood_crimes_dict).set_index("Neighbourhood ID")
neighbourhood_crimes_df["Neighbourhood"] = master_df_sorted["Neighbourhood"].unique()
neighbourhood_crimes_df.sort_values("No. of Crimes", ascending=False, inplace=True)

# create dataframes with top 10 crime (most crime) and bottom 10 (least crime) neighbourhoods 
most_crimes_neighbourhoods = neighbourhood_crimes_df.head(10)
least_crimes_neighbourhoods = neighbourhood_crimes_df.tail(10).sort_values("No. of Crimes").reset_index(drop=True)


In [6]:
# display 10 neighbourhoods with most crimes
most_crimes_neighbourhoods

,Hood Lat,Hood Lng,No. of Crimes,Neighbourhood
Neighbourhood ID,,,,
77,43.644959,-79.387431,9984,Waterfront Communities-The Island (77)
76,43.657212,-79.383738,8765,Bay Street Corridor (76)
75,43.659622,-79.379598,7641,Church-Yonge Corridor (75)
1,43.721569,-79.597207,5759,West Humber-Clairville (1)
73,43.656880,-79.369331,5502,Moss Park (73)
78,43.653612,-79.397825,4551,Kensington-Chinatown (78)
27,43.763923,-79.489869,4131,York University Heights (27)
26,43.727890,-79.498073,4039,Downsview-Roding-CFB (26)
137,43.766636,-79.228039,3854,Woburn (137)


In [7]:
# display 10 neighbourhoods with least crimes
least_crimes_neighbourhoods

,Hood Lat,Hood Lng,No. of Crimes,Neighbourhood
0,43.659076,-79.495543,397,Lambton Baby Point (114)
1,43.694210,-79.312666,398,Woodbine-Lumsden (60)
2,43.714195,-79.480897,408,Maple Leaf (29)
3,43.748982,-79.198807,424,Guildwood (140)
4,43.634161,-79.573765,467,Markland Wood (12)
5,43.694486,-79.337386,493,Old East York (58)
6,43.688566,-79.396944,505,Yonge-St.Clair (97)
7,43.692357,-79.522358,525,Humber Heights-Westmount (8)
8,43.782788,-79.149991,530,Centennial Scarborough (133)
9,43.644921,-79.567969,539,Etobicoke West Mall (13)


In [8]:
# **************************************************************************************************
# add symbols for the most & least dangerous neighbourhood to the heatmap
# **************************************************************************************************

# info box template for pop up message over symbols

info_box_list_most = """
<dl>
<dt>Neighbourhood</dt><dd>{Neighbourhood}</dd>
</dl>
"""

info_box_list_least = """
<dl>
<dt>Neighbourhood</dt><dd>{Neighbourhood}</dd>
</dl>
"""

# Store the DataFrame rows for neighbourhoods
dangerous_neighbourhood_info = [info_box_list_most.format(**row) for index, row in most_crimes_neighbourhoods.iterrows()]
most_crime_locations = most_crimes_neighbourhoods[["Hood Lat", "Hood Lng"]]
dangerous_names = most_crimes_neighbourhoods["Neighbourhood"].to_list()

safe_neighbourhood_info = [info_box_list_least.format(**row) for index, row in least_crimes_neighbourhoods.iterrows()]
least_crime_locations = least_crimes_neighbourhoods[["Hood Lat", "Hood Lng"]]
safe_names = least_crimes_neighbourhoods["Neighbourhood"].to_list()

In [9]:
# create symbol layers for top 10 most dangerous neighourhoods and top 10 most safe neighbourhoods 
most_crime_symbols = gmaps.symbol_layer(most_crime_locations, scale=5, fill_color="red", stroke_color="black", 
                                        hover_text=dangerous_names, info_box_content=info_box_list_most)


least_crime_symbols = gmaps.symbol_layer(least_crime_locations, scale=5, fill_color='blue', stroke_color='green', 
                                         hover_text=safe_names, info_box_content=info_box_list_least)

# add symbol layers to the heatmap
fig.add_layer(most_crime_symbols)
fig.add_layer(least_crime_symbols)
fig

Figure(layout=FigureLayout(height='420px'))

 - Black/red symbols indicate crime hotspots - the most dangerous neighbourhoods in the city of Toronto.
 - Green symbols indicate the top 10 neighbourhoods with the least crimes, and therefore the most safe.
 
### Observations
 - 60% of the top 10 most dangerous neighbours appear to be in the Toronto downtown area, with the rest spread quite far away from each other.
 - The Yonge-St.Clair neighbourhood  is considered a safe neighbourhood with less crimes - this is interesting given its proximity to downtown Toronto where crime cases are high.
 - There appear to be a marginally higher number of safer neighbourhoods in the west end of Toronto.
 

In [10]:
# **************************************************************************************************
# find the geocoordinates of Toronto police stations in the top 10 neighbourhoods with most crimes
# to add as markers on heatmap
# **************************************************************************************************

# create a column with initially null values for police station and its location
most_crimes_neighbourhoods["Police Station"] = np.nan
most_crimes_neighbourhoods["Police Station Lat"] = np.nan
most_crimes_neighbourhoods["Police Station Lng"] = np.nan
     
# set up parameters for querying Google Places API 
target_search = "police"
target_radius = 5000
target_type = "police"

# set up a parameters dictionary 
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# --- define base url ---
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
most_crimes_neighbourhoods



/Users/nehanayeem/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/nehanayeem/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/nehanayeem/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

,Hood Lat,Hood Lng,No. of Crimes,Neighbourhood,Police Station,Police Station Lat,Police Station Lng
Neighbourhood ID,,,,,,,
77,43.644959,-79.387431,9984,Waterfront Communities-The Island (77),NaN,NaN,NaN
76,43.657212,-79.383738,8765,Bay Street Corridor (76),NaN,NaN,NaN
75,43.659622,-79.379598,7641,Church-Yonge Corridor (75),NaN,NaN,NaN
1,43.721569,-79.597207,5759,West Humber-Clairville (1),NaN,NaN,NaN
73,43.656880,-79.369331,5502,Moss Park (73),NaN,NaN,NaN
78,43.653612,-79.397825,4551,Kensington-Chinatown (78),NaN,NaN,NaN
27,43.763923,-79.489869,4131,York University Heights (27),NaN,NaN,NaN
26,43.727890,-79.498073,4039,Downsview-Roding-CFB (26),NaN,NaN,NaN
137,43.766636,-79.228039,3854,Woburn (137),NaN,NaN,NaN


In [11]:
# use a for loop to go through each neighbourhood in the dataframe and make an API call 

for index, row in most_crimes_neighbourhoods.iterrows():
    
    # get the neighbourhood coordinates and store in params dictionary 
    lat = row["Hood Lat"]
    lng = row["Hood Lng"]
    params["location"] = f"{lat},{lng}"
    
    # call the API and get police station details for each neighbourhood 
    police_data = requests.get(base_url, params).json()
    
    # use exception handling to store the police station and location for each neighbourhood 
    
    try:
        station_name = police_data["results"][0]["name"]
        location_lat = police_data["results"][0]["geometry"]["location"]["lat"]
        location_lng = police_data["results"][0]["geometry"]["location"]["lng"]
        
        most_crimes_neighbourhoods.loc[index, "Police Station"] = station_name
        most_crimes_neighbourhoods.loc[index, "Police Station Lat"] = location_lat    
        most_crimes_neighbourhoods.loc[index, "Police Station Lng"] = location_lng  
        
        print(f"Police Station found for '{row['Neighbourhood']}'! The nearest station is {station_name}")
        print("------------------------------------------------------------------------------------------------------------------")
              
    except:
        print(f"Could not find information for neighbourhood '{row['Neighbourhood']}'... skipping.")    
        print("------------------------------------------------------------------------------------------------------------------")
        

/Users/nehanayeem/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Police Station found for 'Waterfront Communities-The Island (77)'! The nearest station is Toronto Police Service 52 Division
------------------------------------------------------------------------------------------------------------------
Police Station found for 'Bay Street Corridor (76)'! The nearest station is Toronto Police Service 52 Division
------------------------------------------------------------------------------------------------------------------
Police Station found for 'Church-Yonge Corridor (75)'! The nearest station is Toronto Police Service 52 Division
------------------------------------------------------------------------------------------------------------------
Police Station found for 'West Humber-Clairville (1)'! The nearest station is Toronto Police Service 23 Division
------------------------------------------------------------------------------------------------------------------
Police Station found for 'Moss Park (73)'! The nearest station is Toronto Poli

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the police markers to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Police Station}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
police_info = [info_box_template.format(**row) for index, row in most_crimes_neighbourhoods.iterrows()]
locations = most_crimes_neighbourhoods[["Police Station Lat", "Police Station Lng"]]

In [13]:
# add police station marker layer on top of heat map 
markers = gmaps.marker_layer(locations, info_box_content = police_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# display new dataframe with police info
most_crimes_neighbourhoods

,Hood Lat,Hood Lng,No. of Crimes,Neighbourhood,Police Station,Police Station Lat,Police Station Lng
Neighbourhood ID,,,,,,,
77,43.644959,-79.387431,9984,Waterfront Communities-The Island (77),Toronto Police Service 52 Division,43.654179,-79.389450
76,43.657212,-79.383738,8765,Bay Street Corridor (76),Toronto Police Service 52 Division,43.654179,-79.389450
75,43.659622,-79.379598,7641,Church-Yonge Corridor (75),Toronto Police Service 52 Division,43.654179,-79.389450
1,43.721569,-79.597207,5759,West Humber-Clairville (1),Toronto Police Service 23 Division,43.743720,-79.584475
73,43.656880,-79.369331,5502,Moss Park (73),Toronto Police Service 52 Division,43.654179,-79.389450
78,43.653612,-79.397825,4551,Kensington-Chinatown (78),Toronto Police Service 52 Division,43.654179,-79.389450
27,43.763923,-79.489869,4131,York University Heights (27),Toronto Police Service 31 Division,43.756792,-79.527411
26,43.727890,-79.498073,4039,Downsview-Roding-CFB (26),Toronto Police 12 Division,43.694432,-79.486998
137,43.766636,-79.228039,3854,Woburn (137),Toronto Police Service,43.749603,-79.288540


#### Observations:
- it appears that Toronto Police 52 Division is the nearest police station for 6 of the top 10 neighbourhoods with most crimes. This shows that the neighbourhoods are close to one another and also calls to question whether this particular division might be overwhelmed with crime incidents. It would be interesting to dive deeper and find out whether there is a correlation between crime rates and government funding for each police divisions.